In [19]:
# first download sembleu
# git clone https://github.com/freesunshine0316/sembleu.git

# import sys
# sys.path.insert(0, "path/to/sembleu")

import sys
import sembleu
import os, json, time

from pathlib import Path

from sembleu import src
from sembleu.src import bleu_score
from sembleu.src.bleu_score import corpus_bleu, sentence_bleu, SmoothingFunction, NgramInst

from sembleu.src import amr_graph
from sembleu.src.amr_graph import AMRGraph

import numpy as np

In [39]:
def read_amr(path):
    id_dict = {}
    amrs, id_list, ids, sents = [],[],[],[]
    amr_str = ''

    for line in open(path,'r'):
                
        if line.startswith('#'):
            if line.startswith('# ::id'):
                id = line.strip().split()[2]
                ids.append(id)
                id_dict[id] = len(ids)-1
                id_list.append(id)
            if line.startswith('# ::snt'):
                snt = line[2:].strip().replace('::snt ', '')
                sents.append(snt)
            continue
            
        line = line.strip()
        if line == '':
            if amr_str != '':
                amrs.append(amr_str.strip())
                amr_str = ''
        else:
            amr_str = amr_str + line + ' '

    if amr_str != '':
        amrs.append(amr_str.strip())
        amr_str = ''
        
    return (amrs, ids, sents), id_list

def get_amr_ngrams(path, stat_save_path=None, raw=False, id_list_tmp=list()):
    
    data, raw_data, sentences = [],[],[]

    if stat_save_path:
        f = open(stat_save_path, 'w')

    (lines, amrids, sents), id_list = read_amr(path) 
    
    if len(amrids)==0:
        amrids = id_list_tmp
        
    for line, amrid, snt in zip(lines, amrids, sents):
        
        if line:
            raw_data.append(line)
            sentences.append(snt)
            try:
                amr = AMRGraph(line.strip())
            except AssertionError:
                assert False
            amr.revert_of_edges()
            ngrams = amr.extract_ngrams(3, multi_roots=True) # dict(list(tuple))
            data.append(NgramInst(ngram=ngrams, length=len(amr.edges)))
            if stat_save_path:
                print(len(amr), len(ngrams[1]), len(ngrams[2]), len(ngrams[3]), file=f)

    if stat_save_path:
        f.close()
    if raw:
        return data, raw_data, sentences, id_list
    else:
        return data, id_list

In [44]:
amr_path = Path('sample-amr')

data, raw_data, sentences, id_list = get_amr_ngrams(amr_path, raw=True)

In [45]:
raw_data[0]

'(t / turn-02 :ARG0 (i2 / it) :ARG1 (l / land :quant (m / multiple :op1 (a / area-quantity :quant 100 :unit (s / square-mile))) :location (s2 / state :wiki "Bihar" :name (n / name :op1 "Bihar"))) :ARG2 (l2 / lake :mod (g / giant)))'

In [46]:
data[0]

NgramInst(ngram={1: [('turn-02',), ('it',), ('land',), ('lake',), ('multiple',), ('state',), ('giant',), ('area-quantity',), ('bihar',), ('name',), ('100',), ('square-mile',), ('bihar',)], 2: [('turn-02', ':ARG0', 'it'), ('turn-02', ':ARG1', 'land'), ('turn-02', ':ARG2', 'lake'), ('land', ':quant', 'multiple'), ('land', ':location', 'state'), ('lake', ':mod', 'giant'), ('multiple', ':op1', 'area-quantity'), ('state', ':wiki', 'bihar'), ('state', ':name', 'name'), ('area-quantity', ':quant', '100'), ('area-quantity', ':unit', 'square-mile'), ('name', ':op1', 'bihar')], 3: [('turn-02', ':ARG1', 'land', ':quant', 'multiple'), ('turn-02', ':ARG1', 'land', ':location', 'state'), ('turn-02', ':ARG2', 'lake', ':mod', 'giant'), ('land', ':quant', 'multiple', ':op1', 'area-quantity'), ('land', ':location', 'state', ':wiki', 'bihar'), ('land', ':location', 'state', ':name', 'name'), ('multiple', ':op1', 'area-quantity', ':quant', '100'), ('multiple', ':op1', 'area-quantity', ':unit', 'square-mil